In [1]:
import json
import random

from pprint import pprint

from datasets import Dataset, load_dataset
from tqdm.notebook import tqdm

In [2]:
SYSTEM_PROMPT = \
"""
Ты — ассистент, который помогает человеку делать суммаризацию текста.\
Твоя задача — формулировать краткое изложение на русском языке, \
точно передавая смысл без искажений и отклонений от оригинала.
""".strip()

In [3]:
def dataset_to_jsonl(
    dataset: Dataset,
    system_prompt: str,
    prompt_column: str,
    response_column: str,
) -> list[dict]:
    lines = []
    for sample in tqdm(dataset):
        item = {
            "messages": [
                {"role": "system", "content": system_prompt.strip()},
                {"role": "user", "content": sample[prompt_column].strip()},
                {"role": "assistant", "content": sample[response_column].strip()}
            ]
        }
        lines.append(item)
    
    return lines

In [4]:
train_dataset = load_dataset("RussianNLP/Mixed-Summarization-Dataset", split="train")
test_dataset = load_dataset("RussianNLP/Mixed-Summarization-Dataset", split="test")

In [5]:
train_dataset = train_dataset.shuffle(seed=42).select(range(2000))

In [6]:
print(train_dataset)

Dataset({
    features: ['text', 'summary'],
    num_rows: 2000
})


In [7]:
prompt_column="text"
response_column="summary"

train_jsonl = dataset_to_jsonl(
    train_dataset,
    system_prompt=SYSTEM_PROMPT,
    prompt_column=prompt_column,
    response_column=response_column
)

test_jsonl = dataset_to_jsonl(
    test_dataset,
    system_prompt=SYSTEM_PROMPT,
    prompt_column=prompt_column,
    response_column=response_column
)

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/258 [00:00<?, ?it/s]

In [8]:
pprint(train_jsonl[0])

{'messages': [{'content': 'Ты — ассистент, который помогает человеку делать '
                          'суммаризацию текста.Твоя задача — формулировать '
                          'краткое изложение на русском языке, точно передавая '
                          'смысл без искажений и отклонений от оригинала.',
               'role': 'system'},
              {'content': 'Дай краткое изложением этого текста.\n'
                          ' Вам понадобятся 2 чайные ложки чайных листьев '
                          'сорта Кашмир, пол чайной ложки бикарбоната соды '
                          '(пищевой соды), 2 раздробленных стручка кардамона, '
                          '2 стакана воды, 2 стакана цельного молока и пол '
                          'чайной ложки морской соли.  Если у вас нет чая '
                          'Кашмир, можно заменить его на обычный зеленый. В '
                          'качестве украшения можно добавить в напиток 1,5 '
                          'чайные ложки фисташ

In [9]:
pprint(test_jsonl[0])

{'messages': [{'content': 'Ты — ассистент, который помогает человеку делать '
                          'суммаризацию текста.Твоя задача — формулировать '
                          'краткое изложение на русском языке, точно передавая '
                          'смысл без искажений и отклонений от оригинала.',
               'role': 'system'},
              {'content': 'Кратко суммаризируй текст:\n'
                          '"Если девушка выбирает свой первый мотоцикл, нужно '
                          'учесть множество разных факторов. Необходимо '
                          'ориентироваться на то, является ли удобным '
                          'положение за рулём. Желательно взять ТС на '
                          'несколько часов и протестировать, чтобы убедиться в '
                          'отсутствии проблем. Хорошо, если можно опробовать '
                          'несколько разных мотоциклов и для себя понять, '
                          'какая между ними разница.\n'
       

In [10]:
def split_train_val(
    data: list,
    train_ratio: float = 0.8,
    seed: int = 42
):
    assert 0.0 < train_ratio < 1.0, "train_ratio должен быть между 0 и 1 (не включая границы)"

    rng = random.Random(seed)
    indices = list(range(len(data)))
    rng.shuffle(indices)

    split_idx = int(len(data) * train_ratio)

    train_indices = indices[:split_idx]
    val_indices  = indices[split_idx:]

    train_list = [data[i] for i in train_indices]
    val_list  = [data[i] for i in val_indices]

    return train_list, val_list

In [11]:
train_jsonl, val_jsonl = split_train_val(train_jsonl)

In [12]:
def write_to_jsonl_file(file_path: str, data: list[dict]):
    with open(file_path, "w") as file:
        for messages in tqdm(data):
            json_line = json.dumps(messages, ensure_ascii=False)
            file.write(json_line + "\n")

In [13]:
write_to_jsonl_file("../../data/train.jsonl", train_jsonl)

  0%|          | 0/1600 [00:00<?, ?it/s]

In [14]:
write_to_jsonl_file("../../data/val.jsonl", val_jsonl)

  0%|          | 0/400 [00:00<?, ?it/s]

In [15]:
write_to_jsonl_file("../../data/test.jsonl", test_jsonl)

  0%|          | 0/258 [00:00<?, ?it/s]